# Candlestick Chart with VP Strategy Signals, SL/TP, and PnL Stats

Loads `data.csv`, runs the Volume Profile breakout strategy (Backtrader), overlays BUY/SELL markers, SL/TP lines, and computes PnL for each trade.

In [ ]:
!pip install mplfinance plotly pandas backtrader

In [ ]:
import pandas as pd
import mplfinance as mpf
import plotly.graph_objects as go
import backtrader as bt
import matplotlib.pyplot as plt

## Load the CSV

In [ ]:
df = pd.read_csv("data.csv", parse_dates=["datetime"])
df.set_index("datetime", inplace=True)
df.head()

## Backtrader Strategy (records signals + SL/TP)

In [ ]:
signals = []

class VPStrategy(bt.Strategy):
    params = dict(tp_rr=2.0)
    def next(self):
        vah = max(self.data.high.get(size=24))
        val = min(self.data.low.get(size=24))
        vwap = (self.data.high[0] + self.data.low[0] + self.data.close[0]) / 3

        if not self.position:
            if self.data.close[0] > vah and self.data.close[0] > vwap:
                sl = val
                tp = self.data.close[0] + (self.data.close[0] - sl) * self.p.tp_rr
                self.buy()
                signals.append((self.data.datetime.datetime(0), self.data.close[0], "BUY", sl, tp))
            elif self.data.close[0] < val and self.data.close[0] < vwap:
                sl = vah
                tp = self.data.close[0] - (sl - self.data.close[0]) * self.p.tp_rr
                self.sell()
                signals.append((self.data.datetime.datetime(0), self.data.close[0], "SELL", sl, tp))

# Run backtest
data = bt.feeds.PandasData(dataname=df)
cerebro = bt.Cerebro()
cerebro.addstrategy(VPStrategy)
cerebro.adddata(data)
cerebro.run()

## Calculate Trade PnL

In [ ]:
trade_results = []

for d,p,s,sl,tp in signals:
    exit_price = None
    # Simple simulation: check which level (SL/TP) would have been hit first
    candle = df.loc[d]
    if s == "BUY":
        if candle['low'] <= sl:
            exit_price = sl
        else:
            exit_price = tp
    else:  # SELL
        if candle['high'] >= sl:
            exit_price = sl
        else:
            exit_price = tp
    pnl = (exit_price - p) if s=="BUY" else (p - exit_price)
    trade_results.append({
        "datetime": d,
        "type": s,
        "entry": p,
        "exit": exit_price,
        "pnl": pnl
    })

df_trades = pd.DataFrame(trade_results)

total_trades = len(df_trades)
winning_trades = len(df_trades[df_trades['pnl']>0])
losing_trades = len(df_trades[df_trades['pnl']<=0])
total_pnl = df_trades['pnl'].sum()

print(f"Total trades: {total_trades}")
print(f"Winning trades: {winning_trades}")
print(f"Losing trades: {losing_trades}")
print(f"Total PnL: {total_pnl:.5f}\n")

df_trades.head(10)

## Static Candlestick (mplfinance) with Signals + SL/TP

In [ ]:
df_plot = df.tail(200).copy()
apds = []

for d,p,s,sl,tp in signals:
    if d in df_plot.index:
        if s == "BUY":
            apds.append(mpf.make_addplot([p], type='scatter', markersize=100, marker='^', color='g', panel=0))
        else:
            apds.append(mpf.make_addplot([p], type='scatter', markersize=100, marker='v', color='r', panel=0))
        df_plot.loc[:, f"SL_{d}"] = sl
        df_plot.loc[:, f"TP_{d}"] = tp
        apds.append(mpf.make_addplot(df_plot[f"SL_{d}"], color='red', linestyle='--'))
        apds.append(mpf.make_addplot(df_plot[f"TP_{d}"], color='green', linestyle='--'))

mpf.plot(df_plot, type="candle", style="yahoo", volume=True, mav=(20,50), addplot=apds)

## Interactive Candlestick (Plotly) with Signals + SL/TP

In [ ]:
fig = go.Figure(data=[go.Candlestick(
    x=df.index,
    open=df["open"],
    high=df["high"],
    low=df["low"],
    close=df["close"]
)])

for d,p,s,sl,tp in signals:
    color = 'green' if s=="BUY" else 'red'
    symbol = 'triangle-up' if s=="BUY" else 'triangle-down'
    fig.add_trace(go.Scatter(x=[d], y=[p], mode="markers", marker_symbol=symbol, marker_size=12, marker_color=color, name=s))
    fig.add_hline(y=sl, line=dict(color='red', dash='dot'))
    fig.add_hline(y=tp, line=dict(color='green', dash='dot'))

fig.update_layout(xaxis_rangeslider_visible=False, template="plotly_dark")
fig.show()